In [ ]:
import numpy as np
import time
import pandas as pd
from scipy.sparse import random as sparse_random
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from numpy.linalg import norm, eigvalsh, eigvals

np.random.seed(42)

resultados = []
# Configura tus tamaños de matriz aquí
n_densos = [100, 500, 1000, 3000]
n_dispersa = [1000, 3000, 5000, 10000]

for tipo in ['densa aleatoria', 'simétrica densa']:
    if tipo == 'densa aleatoria':
        sizes = n_densos
    else:
        sizes = n_densos
    for n in sizes:
        print(f"\nMatriz tipo: {tipo}, tamaño: {n}")
        if tipo == 'densa aleatoria':
            A = np.random.randn(n, n)
            # QR (eigvals)
            t0 = time.time()
            true_eigs = eigvals(A)
            t1 = time.time()
            lambda_real = np.max(np.abs(true_eigs))
            tiempo_qr = t1 - t0
            # Potencias
            b = np.random.rand(n)
            b = b / norm(b)
            t0 = time.time()
            for k in range(1, 151):
                b = np.dot(A, b)
                b = b / norm(b)
                est = np.dot(b, np.dot(A, b))
            t1 = time.time()
            lambda_pot = est
            tiempo_pot = t1 - t0
            error_pot = abs(lambda_pot - lambda_real) / abs(lambda_real)
            resultados.append([tipo, n, tiempo_pot, tiempo_qr, error_pot])
        elif tipo == 'simétrica densa':
            A = np.random.randn(n, n)
            A = (A + A.T) / 2
            t0 = time.time()
            true_eigs = eigvalsh(A)
            t1 = time.time()
            lambda_real = np.max(true_eigs)
            tiempo_qr = t1 - t0
            b = np.random.rand(n)
            b = b / norm(b)
            t0 = time.time()
            for k in range(1, 151):
                b = np.dot(A, b)
                b = b / norm(b)
                est = np.dot(b, np.dot(A, b))
            t1 = time.time()
            lambda_pot = est
            tiempo_pot = t1 - t0
            error_pot = abs(lambda_pot - lambda_real) / abs(lambda_real)
            resultados.append([tipo, n, tiempo_pot, tiempo_qr, error_pot])

for tipo in ['dispersa simétrica']:
    sizes = n_dispersa
    for n in sizes:
        print(f"\nMatriz tipo: {tipo}, tamaño: {n}")
        density = 0.0003
        S = sparse_random(n, n, density=density, format='csr', dtype=np.float64, random_state=42)
        A = (S + S.T) / 2
        t0 = time.time()
        lambda_sparse, _ = eigsh(A, k=1, which='LA')
        t1 = time.time()
        lambda_real = lambda_sparse[0]
        tiempo_qr = t1 - t0
        b = np.random.rand(n)
        b = b / norm(b)
        t0 = time.time()
        for k in range(1, 41):  # menos iteraciones porque dispersa y rápida
            b = A.dot(b)
            b = b / norm(b)
            est = b.dot(A.dot(b))
        t1 = time.time()
        lambda_pot = est
        tiempo_pot = t1 - t0
        error_pot = abs(lambda_pot - lambda_real) / abs(lambda_real)
        resultados.append([tipo, n, tiempo_pot, tiempo_qr, error_pot])

df2 = pd.DataFrame(resultados, columns=['Tipo matriz', 'n', 'Tiempo Potencias (s)', 'Tiempo QR/ARPACK (s)', 'Error relativo Potencias'])
print(df2)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set(style="whitegrid")

# Ejemplo de estructura de datos (debes reemplazar por tu df2 real)
# df2 = pd.DataFrame({
#     'Tipo matriz': ['densa aleatoria', ...],
#     'n': [...],
#     'Tiempo Potencias (s)': [...],
#     'Tiempo QR/ARPACK (s)': [...]
# })

tipos = ['densa aleatoria', 'simétrica densa', 'dispersa simétrica']

fig = plt.figure(constrained_layout=True, figsize=(10, 8))
gs = fig.add_gridspec(2, 2, width_ratios=[1,1.15])  # Segunda columna ligeramente más ancha si quieres

# Primer gráfico: densa aleatoria (arriba izquierda)
ax0 = fig.add_subplot(gs[0,0])
sel = df2['Tipo matriz'] == 'densa aleatoria'
x = df2.loc[sel, 'n']
y1 = df2.loc[sel, 'Tiempo Potencias (s)']
ax0.plot(x, y1, marker='o', color='#4e79a7', label="Potencias")
y2 = df2.loc[sel, 'Tiempo QR/ARPACK (s)']
ax0.plot(x, y2, marker='s', color='#f28e2b', linestyle='--', label="QR/ARPACK")
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_title("Densa aleatoria")
ax0.set_xlabel("Tamaño de matriz $n$")
ax0.set_ylabel("Tiempo (s)")
ax0.legend(fontsize=10)

# Segundo gráfico: simétrica densa (abajo izquierda)
ax1 = fig.add_subplot(gs[1,0])
sel = df2['Tipo matriz'] == 'simétrica densa'
x = df2.loc[sel, 'n']
y1 = df2.loc[sel, 'Tiempo Potencias (s)']
ax1.plot(x, y1, marker='o', color='#4e79a7', label="Potencias")
y2 = df2.loc[sel, 'Tiempo QR/ARPACK (s)']
ax1.plot(x, y2, marker='s', color='#f28e2b', linestyle='--', label="QR/ARPACK")
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title("Simétrica densa")
ax1.set_xlabel("Tamaño de matriz $n$")
ax1.set_ylabel("Tiempo (s)")
ax1.legend(fontsize=10)

# Tercer gráfico: dispersa simétrica (columna derecha ocupando las dos filas)
ax2 = fig.add_subplot(gs[:,1])  # Ocupa ambas filas de la segunda columna
sel = df2['Tipo matriz'] == 'dispersa simétrica'
x = df2.loc[sel, 'n']
y1 = df2.loc[sel, 'Tiempo Potencias (s)']
ax2.plot(x, y1, marker='o', color='#4e79a7', label="Potencias")
# Normalmente no se incluye QR para dispersa simétrica
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_title("Dispersa simétrica")
ax2.set_xlabel("Tamaño de matriz $n$")
ax2.set_ylabel("Tiempo (s)")
ax2.legend(fontsize=10)

plt.tight_layout()
plt.savefig("figure10.png", dpi=300)
plt.show()
